<a href="https://colab.research.google.com/github/rodrigoataidealves/TCC_MBA_USP/blob/main/Codigo_python_TCC_MBA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [60]:
###código validado em 02_10_25_tempo para rodar: 2h

# ============================================================
# Consulta ao Tesouro Data Lake: RREO - Anexo I (RREO-Anexo 01)
# Município: João Monlevade/MG (IBGE: 3136207)
# ============================================================

!pip -q install requests pandas openpyxl tqdm

import time, requests, pandas as pd
from tqdm.auto import tqdm
from datetime import datetime
import os
from google.colab import files

# -------------------
# Etapa 1: Configurações
# -------------------
BASE = "https://apidatalake.tesouro.gov.br/ords/siconfi/tt"
ENDPOINT_RREO = f"{BASE}/rreo"
ID_IBGE = 3136207                 # João Monlevade/MG
CO_ESFERA = "M"                   # Município
CO_TIPO_DEMONSTRATIVO = "RREO"
NO_ANEXO = "RREO-Anexo 01"
BIMESTRES = list(range(1,7))      # 1..6
SLEEP = 1.1                       # intervalo entre requisições
ANO_INICIO = 2013
ANO_FIM = datetime.now().year

print("OK: Configurações definidas.")

# -------------------
# Etapa 2: Função utilitária com paginação
# -------------------
def get_ords(url, params, limit=5000):
    items, offset = [], 0
    while True:
        q = params.copy()
        q.update({"limit": limit, "offset": offset})
        r = requests.get(url, params=q, timeout=60)
        if r.status_code != 200:
            raise RuntimeError(f"HTTP {r.status_code} -> {r.text[:200]}")
        data = r.json()
        if "items" not in data:
            raise RuntimeError(f"Resposta inesperada: {list(data.keys())}")
        items.extend(data["items"])
        if not data.get("hasMore", False):
            break
        offset = int(data.get("offset", 0)) + int(data.get("limit", limit))
        time.sleep(SLEEP)
    return items

print("OK: Função get_ords criada.")

# -------------------
# Etapa 3: Descobrir anos válidos
# -------------------
def ano_tem_rreo(ano:int)->bool:
    params = {
        "an_exercicio": ano,
        "nr_periodo": 1,
        "co_tipo_demonstrativo": CO_TIPO_DEMONSTRATIVO,
        "id_ente": ID_IBGE,
        "no_anexo": NO_ANEXO,
        "co_esfera": CO_ESFERA,
    }
    try:
        dados = get_ords(ENDPOINT_RREO, params, limit=1)
        return len(dados) > 0
    except Exception as e:
        print(f"[AVISO] Falha ao testar ano {ano}: {e}")
        return False

anos_validos = []
for ano in tqdm(range(ANO_INICIO, ANO_FIM+1), desc="Testando anos"):
    if ano_tem_rreo(ano):
        anos_validos.append(ano)
    time.sleep(SLEEP)

print("Anos detectados com dados:", anos_validos)
assert len(anos_validos) > 0, "Nenhum ano detectado para este ente/anexo."

# -------------------
# Etapa 4: Coleta completa
# -------------------
registros = []
for ano in tqdm(anos_validos, desc="Coletando por ano"):
    for bim in tqdm(BIMESTRES, leave=False, desc=f"Ano {ano}"):
        params = {
            "an_exercicio": ano,
            "nr_periodo": bim,
            "co_tipo_demonstrativo": CO_TIPO_DEMONSTRATIVO,
            "id_ente": ID_IBGE,
            "no_anexo": NO_ANEXO,
            "co_esfera": CO_ESFERA,
        }
        try:
            itens = get_ords(ENDPOINT_RREO, params, limit=5000)
            for it in itens:
                it["_an_exercicio"] = ano
                it["_nr_periodo"] = bim
            registros.extend(itens)
        except Exception as e:
            print(f"[ERRO] {ano}-{bim}: {e}")
        time.sleep(SLEEP)

print("Total de registros coletados:", len(registros))
assert len(registros) > 0, "Nenhum registro retornado."

# -------------------
# Etapa 5: DataFrame e checagens
# -------------------
df = pd.DataFrame(registros)
print("Dimensão do DataFrame:", df.shape)
display(df.head(10))

print("\nDistribuição por ano:")
print(df.get("an_exercicio", df["_an_exercicio"]).value_counts().sort_index())

print("\nDistribuição por bimestre:")
print(df.get("nr_periodo", df["_nr_periodo"]).value_counts().sort_index())

# -------------------
# Etapa 6: Exportar para Excel
# -------------------
ARQ = "/content/rreo_anexo1_joao_monlevade.xlsx"
with pd.ExcelWriter(ARQ, engine="openpyxl") as xlw:
    df.to_excel(xlw, index=False, sheet_name="dados")

print("Excel salvo em:", ARQ, "| Tamanho (bytes):", os.path.getsize(ARQ))

# Disponibilizar para download no Colab
files.download(ARQ)


OK: Configurações definidas.
OK: Função get_ords criada.


Testando anos:   0%|          | 0/13 [00:00<?, ?it/s]

Anos detectados com dados: [2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025]


Coletando por ano:   0%|          | 0/11 [00:00<?, ?it/s]

Ano 2015:   0%|          | 0/6 [00:00<?, ?it/s]

...

Total de registros coletados: 24073
Dimensão do DataFrame: (24073, 17)


,exercicio,demonstrativo,periodo,periodicidade,instituicao,cod_ibge,uf,populacao,anexo,esfera,rotulo,coluna,cod_conta,conta,valor,_an_exercicio,_nr_periodo
0,2015,RREO,1,B,Prefeitura Municipal de João Monlevade - MG,3136207,MG,78040,RREO-Anexo 01,M,None,PREVISÃO INICIAL,ReceitasExcetoIntraOrcamentarias,RECEITAS (EXCETO INTRA-ORÇAMENTÁRIAS) (I),1.921620e+08,2015,1
1,2015,RREO,1,B,Prefeitura Municipal de João Monlevade - MG,3136207,MG,78040,RREO-Anexo 01,M,None,PREVISÃO ATUALIZADA (a),ReceitasExcetoIntraOrcamentarias,RECEITAS (EXCETO INTRA-ORÇAMENTÁRIAS) (I),1.921620e+08,2015,1
2,2015,RREO,1,B,Prefeitura Municipal de João Monlevade - MG,3136207,MG,78040,RREO-Anexo 01,M,None,No Bimestre (b),ReceitasExcetoIntraOrcamentarias,RECEITAS (EXCETO INTRA-ORÇAMENTÁRIAS) (I),3.287286e+07,2015,1
3,2015,RREO,1,B,Prefeitura Municipal de João Monlevade - MG,3136207,MG,78040,RREO-Anexo 01,M,None,% (b/a),ReceitasExcetoIntraOrcamentarias,RECEITAS (EXCETO INTRA-ORÇAMENTÁRIAS) (I),1.711000e+01,2015,1
4,2015,RREO,1,B,Prefeitura Municipal de João Monlevade - MG,3136207,MG,78040,RREO-Anexo 01,M,None,Até o Bimestre (c),ReceitasExcetoIntraOrcamentarias,RECEITAS (EXCETO INTRA-ORÇAMENTÁRIAS) (I),3.287286e+07,2015,1
5,2015,RREO,1,B,Prefeitura Municipal de João Monlevade - MG,3136207,MG,78040,RREO-Anexo 01,M,None,% (c/a),ReceitasExcetoIntraOrcamentarias,RECEITAS (EXCETO INTRA-ORÇAMENTÁRIAS) (I),1.711000e+01,2015,1
6,2015,RREO,1,B,Prefeitura Municipal de João Monlevade - MG,3136207,MG,78040,RREO-Anexo 01,M,None,SALDO (a-c),ReceitasExcetoIntraOrcamentarias,RECEITAS (EXCETO INTRA-ORÇAMENTÁRIAS) (I),1.592891e+08,2015,1
7,2015,RREO,1,B,Prefeitura Municipal de João Monlevade - MG,3136207,MG,78040,RREO-Anexo 01,M,None,PREVISÃO INICIAL,ReceitasCorrentes,RECEITAS CORRENTES,1.750280e+08,2015,1
8,2015,RREO,1,B,Prefeitura Municipal de João Monlevade - MG,3136207,MG,78040,RREO-Anexo 01,M,None,PREVISÃO ATUALIZADA (a),ReceitasCorrentes,RECEITAS CORRENTES,1.750280e+08,2015,1
9,2015,RREO,1,B,Prefeitura Municipal de João Monlevade - MG,3136207,MG,78040,RREO-Anexo 01,M,None,No Bimestre (b),ReceitasCorrentes,RECEITAS CORRENTES,3.117408e+07,2015,1



Distribuição por ano:
_an_exercicio
2015    2052
2016    2058
2017    2089
2018    2391
2019    2363
2020    2303
2021    2364
2022    2491
2023    2431
2024    2345
2025    1186
Name: count, dtype: int64

Distribuição por bimestre:
_nr_periodo
1    4075
2    4168
3    4204
4    3833
5    3844
6    3949
Name: count, dtype: int64
Excel salvo em: /content/rreo_anexo1_joao_monlevade.xlsx | Tamanho (bytes): 1710848


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>